In [10]:
import numpy as np
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, TextBox, RadioButtons
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
from BPS.synthesis import prior_pdf, synthesise_2_agents_fast
from BPS.synthesis_plots import plot, plot_2_agents, plot_2_agents_fast
sns.set_style("whitegrid")

%load_ext autoreload
%autoreload 2
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
def update_3(val):
    global mu_1
    global mu_2
    global sig_1
    global sig_2
    mu_1 = mu1_slider.val
    mu_2 = mu2_slider.val
    sig_1 = sig1_slider.val
    sig_2 = sig2_slider.val
    agent1 = st.norm(loc=mu_1, scale=sig_1)
    agent2 = st.norm(loc=mu_2, scale=sig_2)
    mix = synthesise_2_agents_fast(cor_slider.val, r_1, r_2, d, agent1, agent2, q1, q2, np.arange(-6, 6, 0.01))
    axs.lines[1].set_ydata(mix)
    axs.lines[2].set_ydata(agent1.pdf(xs))
    axs.lines[3].set_ydata(agent2.pdf(xs))
    fig.canvas.draw_idle()

# Bayesian Predictive Synthesis
Based on ``Bayesian Predictive Synthesis: Forecast Calibration and Combination'' (2018) by Matthew C. Johnson and Mike West. All mistakes (if any are present) are mine and not the authors of the paper.

## Gaussian kernel, 2 agents

Synthesis equation:
$$\pi(y|H) = \int\alpha(y|x)h(x)dx = \left[ \int \alpha_0(x) h(x) dx \right] \pi_0(x) + \sum q_j \left[ \int \alpha_j(y,x_{-j}) \prod_{i\neq j} h_i(x_i) dx_i \right] h_j(y)$$

$$m(x) \sim N(0, 1) $$
$$\pi_0(x) \sim N(0,1) $$

Agent's forecasts are $h(x)$, and $m(x)=E[h(x)]$ that is it's your expectation as to what agent forecast will be.

Snthesis function:
$$a(x) =  exp[-(e_j)^2 / (2 r_1 \delta_j^2)]\left(1 - dexp[-(e_j)^2 / (2 r_2 \delta_j^2)]\right) $$
$$e_j = x_j - \rho x_{-j}$$

In [51]:
mu_1 = 2
sig_1 = 1
mu_2 = 2.5
sig_2 = 1
cor = 0.0
r_1 = 72.13
r_2 = 2.89
d = 1
q1 = 1/2.5
q2 = 1/2.5
prior = st.norm
agent1 = st.norm(loc=mu_1, scale=sig_1)
agent2 = st.norm(loc=mu_2, scale=sig_2)

In [1]:
fig, axs = plt.subplots(1,1, figsize=(12,6))
xs = np.arange(-5, 5, 0.1)
mix = synthesise_2_agents_fast(cor, r_1, r_2, d, agent1, agent2, q1, q2, np.arange(-6, 6, 0.01))
plot_2_agents_fast(axs, xs, mix, agent1, agent2)
plt.subplots_adjust(left=0.25, bottom=0.3)

axcor = plt.axes([0.25, 0.22, 0.65, 0.02], facecolor='white')
ax_mu1 = plt.axes([0.25, 0.19, 0.65, 0.02], facecolor='white')
ax_sig1 = plt.axes([0.25, 0.16, 0.65, 0.02], facecolor='white')
ax_mu2 = plt.axes([0.25, 0.13, 0.65, 0.02], facecolor='white')
ax_sig2 = plt.axes([0.25, 0.10, 0.65, 0.02], facecolor='white')
resetax = plt.axes([0.8, 0.025, 0.1, 0.04])

button = Button(resetax, 'Reset', color="white", hovercolor='0.975')
def reset(event):
    cor_slider.reset()
    mu1_slider.reset()
    mu2_slider.reset()
    sig1_slider.reset()
    sig2_slider.reset()
    
button.on_clicked(reset)

q_ax = fig.add_axes([0.1, 0.8, 0.05, 0.05])
q_box = TextBox(q_ax, "$q_1$:", initial="0.4")

q2_ax = fig.add_axes([0.1, 0.7, 0.05, 0.05])
q2_box = TextBox(q2_ax, "$q_2$:", initial="0.4")

r_ax = fig.add_axes([0.1, 0.6, 0.05, 0.05])
r_box = TextBox(r_ax, "$r_1$:", initial="72.13")

r2_ax = fig.add_axes([0.1, 0.5, 0.05, 0.05])
r2_box = TextBox(r2_ax, "$r_2$:", initial="2.89")

d_ax = fig.add_axes([0.1, 0.4, 0.05, 0.05])
d_box = TextBox(d_ax, "$d$:", initial="1.0")

def q_submit(expr):
    global q1
    q1 = eval(expr)
    assert q1 + q2 <= 1
    mix = synthesise_2_agents_fast(cor_slider.val, r_1, r_2, d, agent1, agent2, q1, q2, np.arange(-6, 6, 0.01))
    axs.lines[1].set_ydata(mix)
    fig.canvas.draw_idle()
    
    
def q2_submit(expr):
    global q2
    q2 = eval(expr)
    assert q1 + q2 <= 1
    mix = synthesise_2_agents_fast(cor_slider.val, r_1, r_2, d, agent1, agent2, q1, q2, np.arange(-6, 6, 0.01))
    axs.lines[1].set_ydata(mix)
    fig.canvas.draw_idle()
    
def r_submit(expr):
    global r_1
    r_1 = eval(expr)
    mix = synthesise_2_agents_fast(cor_slider.val, r_1, r_2, d, agent1, agent2, q1, q2, np.arange(-6, 6, 0.01))
    axs.lines[1].set_ydata(mix)
    fig.canvas.draw_idle()
    
def r2_submit(expr):
    global r_2
    r_2 = eval(expr)
    mix = synthesise_2_agents_fast(cor_slider.val, r_1, r_2, d, agent1, agent2, q1, q2, np.arange(-6, 6, 0.01))
    axs.lines[1].set_ydata(mix)
    fig.canvas.draw_idle()
    
def d_submit(expr):
    global d
    d = eval(expr)
    mix = synthesise_2_agents_fast(cor_slider.val, r_1, r_2, d, agent1, agent2, q1, q2, np.arange(-6, 6, 0.01))
    axs.lines[1].set_ydata(mix)
    fig.canvas.draw_idle()
    

q_box.on_submit(q_submit)
q2_box.on_submit(q2_submit)
r_box.on_submit(r_submit)
r2_box.on_submit(r2_submit)
d_box.on_submit(d_submit)


cor_slider = Slider(ax=axcor, label="Agent correlation", valmin=-1, valmax=1, valinit=0, color="black")
mu1_slider = Slider(ax=ax_mu1, label='Agent 1 mean', valmin=-2.5, valmax=2.5, valinit=2, color="black")
sig1_slider = Slider(ax=ax_sig1, label='Agent 1 std', valmin=0.5, valmax=1.5, valinit=1, color="black")
mu2_slider = Slider(ax=ax_mu2, label='Agent 2 mean', valmin=-2.5, valmax=2.5, valinit=2.5, color="black")
sig2_slider = Slider(ax=ax_sig2, label='Agent 2 std', valmin=0.5, valmax=1.5, valinit=1, color="black")

cor_slider.on_changed(update_3); mu1_slider.on_changed(update_3); sig1_slider.on_changed(update_3); mu2_slider.on_changed(update_3); sig2_slider.on_changed(update_3)
button.on_clicked(reset)

NameError: name 'plt' is not defined